In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Keys, ActionChains
import time, os

profile = 'Profile 2'#Vó
profile = 'Profile 7'#Mineiro
profile = 'Default'#Jaque

options = webdriver.ChromeOptions()
options.add_argument('user-data-dir=C:\\Users\\vinic\\AppData\\Local\\Google\\Chrome\\User Data')
options.add_argument(f'profile-directory={profile}')
##options.add_argument('headless')

driver = webdriver.Chrome(options=options)




In [30]:
import pandas as pd
import re


cores = \
['PRETO', 'AZUL MARINHO', 'AZUL', 'MARROM', 'VERDE OLIVA', 'VERDE', 'WARSKIN BLACK', 'WARSKIN B.', 'WARSKIN', 'CÁQUI', 'CAQUI', 'TAN', 'DESERT' , 'COYOTE', \
'CINZA', 'GRAFITE', 'CAMURÇA', 'CHUMBO', 'WOODLAND'
]

tamanhos = [ \
    '38','39','40','41','42','43','44','45','46','47','48','49','50', \
    'P', 'M', 'GG', 'G', 'XGG', 'EG'

]

df_catalogo = pd.read_excel('catalogo_invictus.xlsx')
'''
#driver.get('https://taticalmilitaria.painel.magazord.com.br/')
#time.sleep(5)
driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[1]/div/span').click()
driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[1]/ul/li[2]/div').click()
driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[1]/ul/li[2]/ul/li[1]/div').click()
por_pagina = driver.find_element(By.CSS_SELECTOR, '[name="perpage"]')

por_pagina.send_keys('500')
'''

for i, row in df_catalogo.iterrows():
    ini = 1542
    if i < ini:
        continue

    if i >= ini+5000:
        break

    print(row['Hierarquia Produto - nome_produto'] + ' - ' + str(row['código EAN']) + ' - Linha:' + str(i))
    ean = str(row['código EAN'])
    ncm = str(row['código NCM']).replace('.', '')
    descricao_produto_generico = row['Hierarquia Produto - nome_produto']
    descricao_produto_completa = row['Hierarquia Produto - nome_produto'].upper()\
        .replace('PRETA', 'PRETO')\
        .replace(' TAM. ', ' ')\
        .replace(' TAM ', ' ')\
        .replace(' TAM Nº ', ' ')\
        .replace(' Nº ', ' ')\
        .replace(' DE ', ' ')\
        .replace(' INVICTUS', '')\
        .replace(' TÁTICO', '')\
        .replace(' TATICO', '')\
        .replace(' TÁTICA', '')\
        .replace(' TATICA', '')


    cor = ''
    for i_cor in cores:
        if descricao_produto_completa.rfind(i_cor) > 0:
            cor = i_cor
            break

    tam = ''
    for i_tam in tamanhos:
        if descricao_produto_completa.find(cor + ' ' + i_tam) > 0:
            tam = i_tam
            break

    filtro_descricao = driver.find_element(By.CSS_SELECTOR, '[name="descricao_completa"]')
    filtro_descricao.clear()
    filtro_descricao.send_keys(descricao_produto_completa)

    driver.find_element(By.XPATH, "//*[text()='Atualizar']").click()
    time.sleep(2)

    #Se não retornou resultado nenhum    
    if len(driver.find_elements(By.CSS_SELECTOR, '.x-grid-row-checker')) == 0:
        continue

    checkbox = driver.find_element(By.CSS_SELECTOR, '.x-grid-row-checker')
    checkbox.click()
    driver.find_element(By.XPATH, "//*[text()='Alterar']").click()
    driver.find_element(By.XPATH, "//*[text()='Alterar Produto (Novo)']").click()
    time.sleep(3)
    
    alterou = False
    controle_paginacao = True
    achou_variacao = False
    while controle_paginacao == True:
        for el in driver.find_elements(By.CSS_SELECTOR, '.sc-fTFjTM.frSKuo'):
            variacao_input = el.find_elements(By.CSS_SELECTOR, '[name="nome"]')[1].get_attribute('value').upper()

            variacao_planilha = ''
            if len(cor) > 0 and len(tam) > 0:
                variacao_planilha = cor + ' - ' + tam
            elif len(cor) > 0:
                variacao_planilha = cor
            elif len(tam) > 0:
                variacao_planilha = tam

            variacao_input    = variacao_input.replace('VERDE OLIVA', 'VERDE').replace('AZUL MARINHO', 'AZUL')
            variacao_planilha = variacao_planilha.replace('VERDE OLIVA', 'VERDE').replace('AZUL MARINHO', 'AZUL')

            if variacao_input == '' or variacao_input == variacao_planilha:
                achou_variacao = True                
                print('Variação achada: ', variacao_planilha)                

                el_ean = el.find_element(By.CSS_SELECTOR, '[name="ean"]')
                if el_ean.get_attribute('value') == '':
                    el_ean.send_keys(ean)
                    alterou = True
                controle_paginacao = False
                break
        if len(driver.find_elements(By.XPATH, "//*[text()='Próximo']")) > 0:
            if driver.find_element(By.XPATH, "//*[text()='Próximo']").get_attribute('disabled'):
                controle_paginacao = False
            else:
                driver.find_element(By.XPATH, "//*[text()='Próximo']").click()
                time.sleep(0.5)

        else:
            controle_paginacao = False

    time.sleep(0.3)
    driver.find_element(By.XPATH, "//*[text()='Tributação e Dimensão']").click()
    time.sleep(0.7)
    
    el_ncm = driver.find_element(By.CSS_SELECTOR, "[name='ncm']")
    if el_ncm.get_attribute('value') == '':
        el_ncm.send_keys(ncm)
        alterou = True

    alterou_dimensao = False    
    el_peso = driver.find_element(By.CSS_SELECTOR, "[name='peso-0']")
    if el_peso.get_attribute('value') == '':
        el_peso.send_keys('0,500')
        alterou_dimensao = True    
    
    el_altura = driver.find_element(By.CSS_SELECTOR, "[name='altura-0']")
    if el_altura.get_attribute('value') == '':
        el_altura.send_keys('7,00')
    
    el_largura = driver.find_element(By.CSS_SELECTOR, "[name='largura-0']")
    if el_largura.get_attribute('value') == '':
        el_largura.send_keys('20,00')
    
    el_comprimento = driver.find_element(By.CSS_SELECTOR, "[name='comprimento-0']")
    if el_comprimento.get_attribute('value') == '':
        el_comprimento.send_keys('30,00')

    if alterou:
        driver.find_element(By.XPATH, "//*[text()='Salvar']").click()
        time.sleep(1.5)
    
    driver.find_element(By.XPATH, "//*[text()='Fechar']").click()
    time.sleep(0.2)

    deu_erro = False
    if len(driver.find_elements(By.XPATH, "//*[text()='Tributação e Dimensão']")):
        deu_erro = True
        alterou = False
    
    if deu_erro:
        with open('erros.txt', 'a') as file:
            file.write(descricao_produto_completa + ' - ERRO\n')
    elif alterou_dimensao:
        with open('erros.txt', 'a') as file:
            file.write(descricao_produto_completa + ' - Dimensão alterada\n')
    elif not achou_variacao:
        with open('erros.txt', 'a') as file:
            file.write(descricao_produto_completa + ' - Variação não encontrada\n')




    checkbox.click()

CALCA INVICTUS PLATOON WARSKIN BLACK 48 - 7908005345900 - Linha:1542
Variação achada:  WARSKIN BLACK - 48
CALCA INVICTUS PLATOON WARSKIN BLACK 50 - 7908005345917 - Linha:1543
Variação achada:  WARSKIN BLACK - 50
CALÇA INVICTUS WAR PRETO 44 - 7908005308035 - Linha:1544
CALÇA FEM. GALAPAGOS HIKER PRETO COM CHUMBO TAM. 36 - 7908005368510 - Linha:1545
CALÇA FEM. GALAPAGOS HIKER PRETO COM CHUMBO TAM. 38 - 7908005368527 - Linha:1546
CALÇA FEM. GALAPAGOS HIKER PRETO COM CHUMBO TAM. 40 - 7908005368534 - Linha:1547
CALÇA FEM. GALAPAGOS HIKER PRETO COM CHUMBO TAM. 42 - 7908005368541 - Linha:1548
CALÇA FEM. GALAPAGOS HIKER PRETO COM CHUMBO TAM. 44 - 7908005368558 - Linha:1549
CALÇA FEM. GALAPAGOS HIKER PRETO COM CHUMBO TAM. 46 - 7908005368565 - Linha:1550
CALÇA MASC GALAPAGOS HIKER PRETO COM CÁQUI TAM. 38 - 7908005368411 - Linha:1551
CALÇA MASC GALAPAGOS HIKER PRETO COM CÁQUI TAM.40 - 7908005368428 - Linha:1552
CALÇA MASC GALAPAGOS HIKER PRETO COM CÁQUI TAM.42 - 7908005368435 - Linha:1553
CALÇA M

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="descricao_completa"]"}
  (Session info: chrome=124.0.6367.203); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60CFE1522+60802]
	(No symbol) [0x00007FF60CF5AC22]
	(No symbol) [0x00007FF60CE17CE4]
	(No symbol) [0x00007FF60CE66D4D]
	(No symbol) [0x00007FF60CE66E1C]
	(No symbol) [0x00007FF60CEACE37]
	(No symbol) [0x00007FF60CE8ABBF]
	(No symbol) [0x00007FF60CEAA224]
	(No symbol) [0x00007FF60CE8A923]
	(No symbol) [0x00007FF60CE58FEC]
	(No symbol) [0x00007FF60CE59C21]
	GetHandleVerifier [0x00007FF60D2E41BD+3217949]
	GetHandleVerifier [0x00007FF60D326157+3488183]
	GetHandleVerifier [0x00007FF60D31F0DF+3459391]
	GetHandleVerifier [0x00007FF60D09B8E6+823622]
	(No symbol) [0x00007FF60CF65FBF]
	(No symbol) [0x00007FF60CF60EE4]
	(No symbol) [

In [ ]:

driver.find_element(By.XPATH, "//*[text()='Próximo']").click()
if driver.find_element(By.XPATH, "//*[text()='Próximo']").get_attribute('disabled'):
    print('x')
else:
    print('y')


In [ ]:
#driver.get(f'https://b2b.invictus.ind.br/search?brand_id=1')
time.sleep(3)

while len(driver.find_elements(By.CSS_SELECTOR, 'main > section.products-filter > div > div > div.products-card.col-md-9 > div.text-center.see-more-products > ul > a')) > 0:
    driver.find_element(By.CSS_SELECTOR, 'main > section.products-filter > div > div > div.products-card.col-md-9 > div.text-center.see-more-products > ul > a').click()
    time.sleep(2)

lista_produtos_html = driver.find_elements(By.CSS_SELECTOR, 'main > section.products-filter > div > div > div.products-card.col-md-9 > div.col-md-4')
                                                             
for produto_html in lista_produtos_html:
    try:
        titulo = produto_html.find_element(By.CSS_SELECTOR, 'div > div.informations > a:nth-child(1) > h3').text
        preco = produto_html.find_element(By.CSS_SELECTOR, ' div > div.informations > p:nth-child(3) > span').text
        print(titulo.strip(),';',preco.strip())
    except:
        continue



In [21]:
import re


for i in range(1150,1170):
    try:
        #driver.execute_script(f'window.open("https://app3.ideris.com.br/Views/Estoque/Edicao.aspx?produtoId=" + {i});')
        driver.get(f'https://app3.ideris.com.br/Views/EstoqueV2/Cadastrar.aspx?produtoV2Id={i}')
        #time.sleep(1)
        while len(driver.find_elements(By.ID, 'ContentPlaceHolder2_txtTitulo')) == 0:
            time.sleep(0.01)
        try:
            titulo = driver.find_element(By.ID, 'ContentPlaceHolder2_txtTitulo').get_attribute('value')
            if driver.find_element(By.ID, 'ContentPlaceHolder2_ddlNcm').get_attribute('value') != '0':
                continue 
        except Exception as err:
            print('Deu erro aqui. i: ', i, '---', err)
            continue
        
        produto_ncm = {
            'MEIA':'MEIA',
            'COLDRE':'COLDRE',
            'CALÇA':'BERMUDA',
            'BERMUDA':'BERMUDA',
            'MALA':'MALA',
            'MOCHILA':'MALA',
            'CAMISETA':'CAMISETA',
            'ABAFADOR':'ABAFADOR',
            'PORTA ALGEMA':'PORTA',
            'BALACLAVA':'BALACLAVA',
            'BANDOLEIRA':'BANDOLEIRA',
            'CAMISA TÁTICA':'CAMISETA',
            'CAMISA':'CAMISETA',
            'CAMISA TÉRMICA':'BLUSA',
            'BOTA':'BOTA',
            'CANETA':'CANETA',
            'CANIVETE':'CANIVETE',
            'MULTITOOL':'CANIVETE',
            'PORTA CARREGADOR':'PORTA',
            'PORTA ALGEMAS':'PORTA',
            'ALGEMA':'ALGEMA',
            'CARREGADOR BATERIA':'CARREGADOR',
            'CARTEIRA':'CARTEIRA',
            'COPO':'COPO',
            'CUECA':'CUECA',
            'FACA':'FACA',
            'FECHADURA ELETRÔNICA':'FECHADURA',
            'GORRO':'GORRO',
            'GARRAFA':'GARRAFA',
            'GUIA MOLA':'GUIA',
            'JAQUETA':'JAQUETA',
            'JOELHEIRA':'JOELHEIRA',
            'LANTERNA':'LANTERNA',
            'LUVA':'LUVA',
            'MACHADO':'MACHADO',
            'SHEMAGH':'SHEMAGH',
            'SACO DESCARTE':'SACO',
            'PALMILHA':'PALMILHA',
            'PATCH':'PATCH',
            'ÓCULOS':'ÓCULOS',
            'REFIL HIDRATAÇÃO':'REFIL',
            'SEGUNDA PELE':'BLUSA',
            'CINTO':'BANDOLEIRA ',
            'CAPA COLETE':'CAPA COLETE ',
        }
        salvar = False
        for produto_ncm_chave, produto_ncm_valor in produto_ncm.items():
            produto_ncm_valor_re = '.*' + produto_ncm_valor.replace(' ','.*') + '.*'

            if re.match(produto_ncm_valor_re, titulo.upper(), re.I):
                driver.find_element(By.ID, 'ContentPlaceHolder2_ddlNcm').send_keys(produto_ncm_valor)
                salvar = True
                continue
        if salvar:
            driver.find_element(By.ID, 'ContentPlaceHolder2_btnSalvar').click()#SALVAR
            time.sleep(0.5)
    except:
        print('Deu Erro:', i)


In [ ]:
"""
import sqlite3
from pathlib import Path


ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME
connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()
"""

driver.get(f'https://www.mercadolivre.com.br/vendas/omni/lista?startPeriod=WITH_DATE_CLOSED_6M_OLD&sort=DATE_CLOSED_DESC')
time.sleep(10)
vendas_lista_html = \
    driver.find_elements(By.CSS_SELECTOR, '#desktop > div.sc-page > div > div.sc-tab-content > div > div.sc-list.sc-list-marketplace > div.andes-card')

for venda_html in vendas_lista_html:
    numero_venda = venda_html.find_element(By.CSS_SELECTOR, 'div > div.identification-row > div.left-column > div.left-column__pack-id').text
    print(numero_venda)

"""        
connection.commit()
cursor.close()
connection.close()
"""


Funções de envio de mensagem e shit_enter(para quebrar linha dentro do input de msg)

In [3]:
def shift_enter(driver):
        ActionChains(driver)\
            .key_down(Keys.SHIFT)\
            .send_keys(Keys.ENTER)\
            .key_up(Keys.SHIFT)\
            .perform()
    

def envia_msg(nome, telefone, msg, enviar=True):
    driver.get(f'https://web.whatsapp.com/send?phone={telefone}')
    
    input_box_msg = WebDriverWait.__class__
    # Esperar para encontrar o input
    input_box_msg = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, '#main > footer > div._2lSWV._3cjY2.copyable-area > div > span:nth-child(2) > div > div._1VZX7 > div._3Uu1_ > div.g0rxnol2.ln8gz9je.lexical-rich-text-input > div.to2l77zo.gfz4du6o.ag5g9lrv.bze30y65.kao4egtt > p')
        )
    )

    input_box_msg.send_keys(f'Olá {nome}')
    #shift_enter(driver)
    if type(msg) == str:
        input_box_msg.send_keys(msg)
    if type(msg) == set:
        for m in msg:
            input_box_msg.send_keys(m[0]+' - '+m[1])
            shift_enter(driver)
            shift_enter(driver)
    if enviar:
        #Enter para enviar msg
        input_box_msg.send_keys(Keys.ENTER)

    time.sleep(1)

In [ ]:
import sqlite3
from pathlib import Path
from selenium.common.exceptions import TimeoutException

ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME

connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()
log_erro = set()
#ler para quem enviar e números
cursor.execute(f'SELECT id, nome, telefone FROM envios')

for row in cursor.fetchall():
    cod, nome, telefone = row
    try:
        compras = set()
        cursor.execute('SELECT produto, vendedor FROM compras')
        for compra in cursor.fetchall():
            produto, vendedor = compra
            compras.add((produto, vendedor,))
        envia_msg(nome, telefone, compras, False)
    except TimeoutException as err:
        cursor.execute(f'UPDATE envios SET log = "Tempo excedido ou telefone inválido" WHERE id = {cod}')

connection.commit()

cursor.close()
connection.close()

In [ ]:
#anexar arquivo

#Clicka no botão +
driver.find_element(By.CSS_SELECTOR, '#main > footer > div._2lSWV._3cjY2.copyable-area > div > span:nth-child(2) > div > div._2xy_p._1bAtO > div > div > div > div > span').click()
time.sleep(1)
#Acha o input de arq
input_file = driver.find_element(By.CSS_SELECTOR, '#main > footer > div._2lSWV._3cjY2.copyable-area > div > span:nth-child(2) > div > div._2xy_p._1bAtO > div > div > span > div > ul > div > div:nth-child(1) > li > div > input[type=file]')
s = os.path.abspath('envios.xlsx') +' '+ os.path.abspath('envios2.xlsx')
print(s)
input_file.send_keys(s)
time.sleep(5)

"""
#Acha o input de arq 2
input_file2 = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div._2QgSC > div._2Ts6i._2xAQV > span > div > span > div > div > div.g0rxnol2.thghmljt.p357zi0d.rjo8vgbg.ggj6brxn.f8m0rgwh.gfz4du6o.r7fjleex.bs7a17vp.ov67bkzj > div > input[type=file]')
input_file2.send_keys(os.path.abspath('envios2.xlsx'))
time.sleep(5)

#Acha o input de arq 3
input_file3 = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div._2QgSC > div._2Ts6i._2xAQV > span > div > span > div > div > div.g0rxnol2.thghmljt.p357zi0d.rjo8vgbg.ggj6brxn.f8m0rgwh.gfz4du6o.r7fjleex.bs7a17vp.ov67bkzj > div > input[type=file]')
input_file3.send_keys(os.path.abspath('envios.xlsx'))
time.sleep(5)
"""

#driver.find_element(By.CSS_SELECTOR, '#app > div > div > div._2QgSC > div._2Ts6i._2xAQV > span > div > span > div > div > div.g0rxnol2.thghmljt.p357zi0d.rjo8vgbg.ggj6brxn.f8m0rgwh.gfz4du6o.r7fjleex.bs7a17vp.ov67bkzj > div > div.O2_ew > div._3wFFT > div > div > span').click()


In [50]:
import sqlite3
from pathlib import Path
from selenium.common.exceptions import TimeoutException

ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME

connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()

cursor.execute(f'DELETE FROM COMPRAS')
cursor.execute(f'DELETE FROM ENVIOS')
cursor.execute(f'INSERT INTO ENVIOS (NOME, TELEFONE) VALUES ("Erro","55479632992") ')
cursor.execute(f'INSERT INTO ENVIOS (NOME, TELEFONE) VALUES ("Gustavo","554796329923") ')


connection.commit()

cursor.close()
connection.close()

In [31]:
ss = 'aa bb cc dd ee'


['aa', 'cc'].count('aa')
ss.count()

TypeError: must be str, not list